In [ ]:
import os
# Check if you're on Google drive or on your own machine.
# Get path to your data.
if ('google' in str(get_ipython())):
    from google.colab import drive
    drive.mount('ME', force_remount=True)
    predir='ME/MyDrive/thesis_final'

In [ ]:

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import seaborn as sns
import pandas as pd
import numpy as np
import warnings
from tqdm import tqdm
import pickle

from sklearn.preprocessing import StandardScaler
import sympy
import torch
import torch.nn as nn


import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [ ]:
import io
class CPU_Unpickler(pickle.Unpickler):
    def find_class(self, module, name):
        if module == 'torch.storage' and name == '_load_from_bytes':
            return lambda b: torch.load(io.BytesIO(b), map_location='cpu')
        else: return super().find_class(module, name)

# notebook variables

In [ ]:
from variables import groups,group_order,groupmap,group_titles,finished_usernames

In [ ]:
with open(predir+'/df_bypost_all.pkl', 'rb') as f:
    df_bypost = pickle.load(f)

df_bypost = df_bypost.dropna(subset=['post_times']).query('likes > 0')[df_bypost['username'].isin(finished_usernames)]

# Extract cyclical time features
def fourier_encode(df):
    df['hour'] = df['post_times'].dt.hour
    df['day_of_week'] = df['post_times'].dt.dayofweek  # 0=Monday
    df['month'] = df['post_times'].dt.month
    df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
    df['day_sin'] = np.sin(2 * np.pi * df['day_of_week'] / 7)
    df['day_cos'] = np.cos(2 * np.pi * df['day_of_week'] / 7)
    df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
    return df

data = fourier_encode(df_bypost)


# Time since last post (in hours)
data['timedelta'] = data['timedelta'].fillna(-1)  # First post

data['log_likes'] = np.log(data['likes']+1)

# Rolling average of likes (past 14 posts)
data['rolling_likes'] = (
    data.groupby('username')['log_likes']
    .transform(lambda x: x.shift(1).rolling(14, min_periods=1).mean())
)

# get the rate of change using np.gradient
data['deriv1'] = np.gradient(data['rolling_likes'])
data['deriv2'] = np.gradient(data['deriv1'])

data = data[data['date'] > pd.to_datetime('2023-01-01')]

df_bypost = df_bypost.query('username != "them"')

In [ ]:
REACHBACK = 14
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# model functions

In [ ]:
def df_to_traintest(df, PrepData):
  dataprepper = PrepData(df, reachback_length = REACHBACK)
  dataset = [x for x in dataprepper.get_dataset()]

  X, y = torch.cat([x[0] for x in dataset], dim=0), torch.tensor([x[1] for x in dataset])
  X = X.reshape(len(dataset), dataprepper.reachback_length*X.shape[1])

  torch.manual_seed(42)
  train, val = torch.utils.data.random_split(list(zip(X,y)), [int(0.8*len(dataset)), len(dataset) - int(0.8*len(dataset))])

  # print(f"Train size: {len(train)}")
  # print(f"Val size: {len(val)}")

  trainx, trainy = np.vstack([x[0] for x in train]), np.array([x[1] for x in train])
  valx, valy = np.vstack([x[0] for x in val]), np.array([x[1] for x in val])

  # Convert numpy arrays to PyTorch datasets
  train_dataset = torch.utils.data.TensorDataset(torch.tensor(trainx, dtype=torch.float32),
                                                torch.tensor(trainy, dtype=torch.float32))
  val_dataset = torch.utils.data.TensorDataset(torch.tensor(valx, dtype=torch.float32),
                                              torch.tensor(valy, dtype=torch.float32))

  # Create data loaders
  batch_size = 64
  train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
  val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size,shuffle=True, num_workers=0)

  return train_loader, val_loader

def data_loader(df,PrepData, reachback_length=REACHBACK):
  dataprepper = PrepData(df, reachback_length=reachback_length)
  dataset = [x for x in dataprepper.get_dataset()]

  X, y = torch.cat([x[0] for x in dataset], dim=0), torch.tensor([x[1] for x in dataset])
  X = X.reshape(len(dataset), dataprepper.reachback_length*X.shape[1])

  dataset = torch.utils.data.TensorDataset(torch.tensor(X, dtype=torch.float32),
                                            torch.tensor(y, dtype=torch.float32))
  batch_size = 64
  loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,shuffle=True, num_workers=12)
  return loader

In [ ]:
class TwoBranchPrepData():
    def __init__(self, df, reachback_length=REACHBACK):
        self.df = df
        self.reachback_length = reachback_length
        self.users = df['username'].unique()

        # Precompute reachback
        self.reachback = []
        for user in self.users:
            user_df = self.df[self.df['username'] == user]
            for i in range(len(user_df) - (self.reachback_length)):
                seq = user_df.iloc[i:i+self.reachback_length]
                target = user_df.iloc[i+self.reachback_length]['likes']
                target = np.log(target+1)
                self.reachback.append((seq, target))

    def __getitem__(self, idx):
        reachback, target = self.reachback[idx]

        # Text embeddings (reachback_length x 384)
        text_features = torch.tensor(
            np.stack(reachback['caption_embedding'].values),
            dtype=torch.float32
        )

        # Time features (reachback_length x 6)
        time_features = torch.tensor(
            reachback[['hour_sin', 'hour_cos', 'day_sin', 'day_cos', 'month_sin', 'month_cos']].values,
            dtype=torch.float32
        )

        # Historical features (reachback_length x 4)
        historical_features = torch.tensor(
            reachback[['rolling_likes', 'timedelta','deriv1','deriv2']].values,
            dtype=torch.float32
        )

        X = torch.cat([text_features, time_features, historical_features], dim=1)
        return X, target

    def get_dataset(self):
        for user in self.users:
            user_df = self.df[self.df['username'] == user]
            for i in range(len(user_df) - self.reachback_length):
                value = self.__getitem__(i)
                # check if there is any nan
                if value[0].isnan().any():
                    continue
                else:
                  yield value

class TwoBranchLikesPredictor(nn.Module):
    def __init__(self, sbert_dim=384, time_dim=6, historical_dim=4,
                 hidden_dim=64, reachback_length=REACHBACK,
                 verobose=False):
        super().__init__()
        self.reachback_length = reachback_length
        self.sbert_dim = sbert_dim
        self.time_dim = time_dim
        self.historical_dim = historical_dim

        self.verbose = verobose

        # Text processing
        self.text_fc = nn.Linear(sbert_dim, 128) # play with output dimension
        self.text_lstm = nn.LSTM(
            input_size=128,
            hidden_size=64,
            batch_first=True
        )

        # LSTM for temporal/historical patterns
        self.lstm = nn.LSTM(
            input_size=time_dim + historical_dim,
            hidden_size=hidden_dim,
            batch_first=True
        )

        # Final prediction
        self.fc = nn.Sequential(
            nn.Linear(64 + hidden_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        # Reshape input: (batch_size, reachback_length * total_features) ->
        # (batch_size, reachback_length, total_features)
        x = x.view(-1, self.reachback_length, self.sbert_dim + self.time_dim + self.historical_dim)

        # Split features
        text_features = x[:, :, :self.sbert_dim]
        time_features = x[:, :, self.sbert_dim:self.sbert_dim+self.time_dim]
        historical_features = x[:, :, self.sbert_dim+self.time_dim:self.sbert_dim+self.time_dim + self.historical_dim]

        if self.verbose:
            print(f"Text features shape: {text_features.shape}")
            print(f"Time features shape: {time_features.shape}")
            print(f"Historical features shape: {historical_features.shape}")

        # Process text (batch_size, reachback_length, 128)
        text_emb = self.text_fc(text_features)
        text_lstm = self.text_lstm(text_emb)
        text_last = text_lstm[0][:, -1, :]  # Last timestep
        # text_agg = torch.mean(text_emb, dim=1)  # Average over sequence

        if self.verbose:
            print(f"text fc: {text_emb.shape}")
            print(f"Text agg shape: {text_last.shape}")

        # Process temporal features (batch_size, reachback_length, 6)
        temporal_input = torch.cat([time_features, historical_features], dim=2)
        lstm_out, _ = self.lstm(temporal_input)
        lstm_last = lstm_out[:, -1, :]  # Last timestep

        if self.verbose:
            print(f"LSTM output shape: {lstm_last.shape}")

        # Combine features
        combined = torch.cat([text_last, lstm_last], dim=1)

        if self.verbose:
            print(f"Combined shape: {combined.shape}")

        return self.fc(combined)

In [ ]:
def train_model(model, train_loader, val_loader, lr=0.001, num_epochs= 10, verbose = False):
  optimizer = torch.optim.Adam(model.parameters(), lr=lr)
  criterion = nn.MSELoss()

  train_loss_history = []
  val_loss_history = []
  for epoch in tqdm(range(num_epochs)):
      # Training phase
      model.train()
      train_loss = 0.0
      for inputs, targets in train_loader:
          inputs = inputs.to(device)
          targets = targets.to(device).float()
          optimizer.zero_grad()

          outputs = model(inputs)
          loss = criterion(outputs.squeeze(), targets)
          loss.backward()
          optimizer.step()

          train_loss += loss.item() * inputs.size(0)

      # Calculate average training loss
      train_loss = train_loss / len(train_loader.dataset)

      # Validation phase
      model.eval()
      val_loss = 0.0
      with torch.no_grad():
          for inputs, targets in val_loader:
              inputs = inputs.to(device)
              targets = targets.to(device).float()
              outputs = model(inputs)
              loss = criterion(outputs.squeeze(), targets)
              val_loss += loss.item() * inputs.size(0)

      val_loss = val_loss / len(val_loader.dataset)

      train_loss_history.append(train_loss)
      val_loss_history.append(val_loss)

      if verbose:
        print(f'Epoch {epoch+1}/{num_epochs}')
        print(f'Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}')
        print('-' * 50)
  return model, train_loss_history, val_loss_history

# Split control

In [ ]:
# split control

split_before = pd.to_datetime('2024-07-01')
data_before = data[data['date'] < split_before]
data_before = data_before[data_before['date'] > pd.to_datetime('2024-02-01')]

split = pd.to_datetime('2024-08-01')
data_after = data[data['date'] >= split]
data_after = data_after[data_after['date'] <= pd.to_datetime('2024-11-01')]

# for the data after, only include [reachback:] for each account
data_after = data_after.groupby('username').apply(lambda x: x.iloc[REACHBACK:]).reset_index(drop=True)

### get residuals

In [ ]:
# two branch

trainval_grouped_nomeaning = {}
for group, usernames in groups.items():
  group_df = data_before.query('group == @group')
  print(group, len(group_df), '-'*30)
  if len(group_df) > 100:
    trainval_grouped_nomeaning[group] = df_to_traintest(group_df, TwoBranchPrepData)
  else:
    print(f'{group} not enough data')

model_grouped_nomeaning= {}
for group, (train_loader, val_loader) in trainval_grouped_nomeaning.items():
  print(f"Training {group}")
  model = TwoBranchLikesPredictor(
      sbert_dim=384,
      reachback_length=REACHBACK,
      verobose=False
  )

  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  # device = torch.device('cpu')
  model = model.to(device)
  print(f"Using device: {device}")

  torch.save(model.state_dict(), predir+f'/models/{group}_{split_before}.pt')

  model, train_loss_history, val_loss_history = train_model(model, train_loader, val_loader, lr=0.001, verbose=False, num_epochs=20)

  model_grouped_nomeaning[group] = (model, train_loss_history, val_loss_history)

In [ ]:
model_grouped_nomeaning = {}
for group in group_order:
  model = TwoBranchLikesPredictor(
      sbert_dim=384,
      reachback_length=REACHBACK,
      verobose=False
  )
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  model = model.to(device)
  if torch.cuda.is_available():
    model.load_state_dict(torch.load(predir+f'/models/{group}_{split_before}.pt'))
  else:
    model.load_state_dict(torch.load(predir+f'/models/{group}_{split_before}.pt', map_location=torch.device('cpu')))
  model_grouped_nomeaning[group] = (model, [],[])

In [ ]:
# residuals_grouped_nomeaning = {}

for group, (model, train_loss_history, val_loss_history) in tqdm(model_grouped_nomeaning.items()):
  usernames = groups[group]
  data_after_group = data_after.query(f'group == @group')
  data_before_group = data_before.query(f'group == @group')
  if len(data_after_group) < 30:
    print(f'{group} not enough after data')
    continue
  if len(data_before_group) < 30:
    print(f'{group} not enough before data')
    continue
  dataloader_after = data_loader(data_after_group, TwoBranchPrepData)
  dataloader_before_train = trainval_grouped_nomeaning[group][0]
  dataloader_before_val = trainval_grouped_nomeaning[group][1]

  model.eval()
  residuals_before = []
  residuals_after = []
  residuals_before_train = []
  residuals_before_val = []

  with torch.no_grad():
      for inputs, targets in dataloader_after:
          inputs, targets = inputs.to(device), targets.to(device)
          outputs = model(inputs)
          residuals_after.extend(targets - outputs.squeeze())

  with torch.no_grad():
    for inputs, targets in dataloader_before_train:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        residuals_before_train.extend(targets - outputs.squeeze())
        residuals_before.extend(targets - outputs.squeeze())

  with torch.no_grad():
    for inputs, targets in dataloader_before_val:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        residuals_before_val.extend(targets - outputs.squeeze())
        residuals_before.extend(targets - outputs.squeeze())

  residuals_grouped_nomeaning[group] = (residuals_before_train,residuals_before_val, residuals_before, residuals_after)

# with open(predir+'/residuals/residuals_grouped_nomeaning.pkl', 'wb') as f:
#   pickle.dump(residuals_grouped_nomeaning, f)

In [ ]:
# turn from torch to numpy array
for group in tqdm(residuals_grouped_nomeaning.keys()):
  # for before in before_dates:
  residuals_before_train,residuals_before_val, residuals_before, residuals_after = residuals_grouped_nomeaning[group]
  residuals_before_train = [float(x) for x in residuals_before_train]
  residuals_before_val = [float(x) for x in residuals_before_val]
  residuals_before = [float(x) for x in residuals_before]
  residuals_after = [float(x) for x in residuals_after]
  residuals_grouped_nomeaning[group] = residuals_before_train,residuals_before_val, residuals_before, residuals_after

with open(predir+'/residuals/residuals_control.pkl', 'wb') as f:
  pickle.dump(residuals_grouped_nomeaning, f)

# split 1

In [ ]:
date_policy1 = pd.to_datetime('2024-02-09')
earliest_before = pd.to_datetime('2023-10-01')

before_dates = pd.date_range(start=earliest_before - pd.Timedelta(days=1), end=date_policy1 - pd.Timedelta(days=14), freq='1ME') + pd.Timedelta(days=1)
len(before_dates), before_dates

### get residuals

In [ ]:
models1 = dict()
residuals1 = dict()

data_after = data[data['date'] >= date_policy1]
# for group in tqdm(group_order):
for group in ['queer']:
  data_after_group = data_after[data_after['group'] == group]

  data_group = data[data['group'] == group]
  for before in before_dates:
    data_before_group = data_group[data_group['date'] < before]
    if len(data_before_group) < 100:
      print(f"Not enough data for {group} before {before}")
    else:
      # dataloaders
      train_loader, val_loader = df_to_traintest(data_before_group, TwoBranchPrepData)
      after_loader = data_loader(data_after_group, TwoBranchPrepData)

      # train
      model = TwoBranchLikesPredictor(
          sbert_dim = 384, reachback_length=REACHBACK
      ).to(device)

      model, train_loss_history, val_loss_history = train_model(model, train_loader, val_loader, lr=0.001, num_epochs= 20, verbose = False)

      torch.save(model.state_dict(), f'{predir}/models/{group}_{before}.pt')

      # get residuals
      model.eval()
      residuals_train, residuals_val, residuals_before, residuals_after = [], [], [], []
      with torch.no_grad():
          for inputs, targets in train_loader:
              inputs, targets = inputs.to(device), targets.to(device)
              outputs = model(inputs)
              residuals_train.extend(targets - outputs.squeeze())
              residuals_before.extend(targets - outputs.squeeze())

      with torch.no_grad():
          for inputs, targets in val_loader:
              inputs, targets = inputs.to(device), targets.to(device)
              outputs = model(inputs)
              residuals_val.extend(targets - outputs.squeeze())
              residuals_before.extend(targets - outputs.squeeze())

      with torch.no_grad():
          for inputs, targets in after_loader:
              inputs, targets = inputs.to(device), targets.to(device)
              outputs = model(inputs)
              residuals_after.extend(targets - outputs.squeeze())

      group_residuals = (residuals_train, residuals_val, residuals_before, residuals_after)
      with open(f'{predir}/residuals/{group}_{before}.pkl', 'wb') as f:
        pickle.dump(group_residuals, f)

      # residuals1[group][before] = (residuals_train, residuals_val, residuals_before, residuals_after)


In [ ]:
residuals1 = dict()
for group in tqdm(group_order):
  residuals1[group] = dict()
  for before in before_dates:
    try:
      if torch.cuda.is_available():
        with open(f'{predir}/residuals/{group}_{before}.pkl', 'rb') as f:
          residuals1[group][before] = pickle.load(f)
      else:
        with open(f'{predir}/residuals/{group}_{before}.pkl', 'rb') as f:
          residuals1[group][before] = CPU_Unpickler(f).load()
    except:
      print(f"No residuals for {group} before {before}")
      residuals1[group][before] = [[0],[0],[0],[0]]


In [ ]:
for group in tqdm(group_order):
  for before in before_dates:
    try:
      residuals_train, residuals_val, residuals_before, residuals_after = residuals1[group][before]
      # move to cpu, change to numpy array
      residuals_train = [float(x.cpu()) for x in residuals_train]
      residuals_val = [float(x.cpu()) for x in residuals_val]
      residuals_before = [float(x.cpu()) for x in residuals_before]
      residuals_after = [float(x.cpu()) for x in residuals_after]
      residuals1[group][before] = (residuals_train, residuals_val, residuals_before, residuals_after)
    except:
      print(f"No residuals for {group} before {before}")
      residuals1[group][before] = [[0],[0],[0],[0]]


In [ ]:
with open(f'{predir}/residuals/residuals1.pkl', 'wb') as f:
  pickle.dump(residuals1, f)

## Split 1a

In [ ]:
date_policy1 = pd.to_datetime('2024-02-09')
earliest_before = pd.to_datetime('2023-10-1')

before_dates1a = pd.date_range(start=earliest_before - pd.Timedelta(days=1), end=date_policy1 - pd.Timedelta(days=14), freq='7D')[:4] + pd.Timedelta(days=1)
before_dates1a = before_dates1a[1:] # 0th value is the same as split 1, can use the same models and residuals
len(before_dates1a), before_dates1a

#### getting residuals

In [ ]:
data_after = data[data['date'] >= date_policy1]
for group in tqdm(['zionist','palestine']):
  data_after_group = data_after[data_after['group'] == group]

  data_group = data[data['group'] == group]
  for before in before_dates1a[:2]:
    data_before_group = data_group[data_group['date'] < before]
    if len(data_before_group) < 100:
      print(f"Not enough data for {group} before {before}")
    else:
      # dataloaders
      train_loader, val_loader = df_to_traintest(data_before_group, TwoBranchPrepData)
      after_loader = data_loader(data_after_group, TwoBranchPrepData)

      # train
      model = TwoBranchLikesPredictor(
          sbert_dim = 384, reachback_length=REACHBACK
      ).to(device)

      model, train_loss_history, val_loss_history = train_model(model, train_loader, val_loader, lr=0.001, num_epochs= 20, verbose = False)

      torch.save(model.state_dict(), f'{predir}/models/{group}_{before}_a.pt')

      # get residuals
      model.eval()
      residuals_train, residuals_val, residuals_before, residuals_after = [], [], [], []
      with torch.no_grad():
          for inputs, targets in train_loader:
              inputs, targets = inputs.to(device), targets.to(device)
              outputs = model(inputs)
              residuals_train.extend(targets - outputs.squeeze())
              residuals_before.extend(targets - outputs.squeeze())

      with torch.no_grad():
          for inputs, targets in val_loader:
              inputs, targets = inputs.to(device), targets.to(device)
              outputs = model(inputs)
              residuals_val.extend(targets - outputs.squeeze())
              residuals_before.extend(targets - outputs.squeeze())

      with torch.no_grad():
          for inputs, targets in after_loader:
              inputs, targets = inputs.to(device), targets.to(device)
              outputs = model(inputs)
              residuals_after.extend(targets - outputs.squeeze())

      group_residuals = (residuals_train, residuals_val, residuals_before, residuals_after)
      with open(f'{predir}/residuals/{group}_{before}_a.pkl', 'wb') as f:
        pickle.dump(group_residuals, f)

In [ ]:
residuals1a = dict()
for group in tqdm(['palestine','zionist']):
  residuals1a[group] = dict()
  for before in before_dates1a:
    try:
      with open(f'{predir}/residuals/{group}_{before}_a.pkl', 'rb') as f:
        residuals1a[group][before] = pickle.load(f)
    except:
      print(f"No residuals for {group} before {before}")
      residuals1a[group][before] = [[0],[0],[0],[0]]

In [ ]:
for group in tqdm(['palestine','zionist']):
  for before in before_dates1a:
    try:
      residuals_train, residuals_val, residuals_before, residuals_after = residuals1a[group][before]
      # move to cpu, change to numpy array
      residuals_train = [float(x.cpu()) for x in residuals_train]
      residuals_val = [float(x.cpu()) for x in residuals_val]
      residuals_before = [float(x.cpu()) for x in residuals_before]
      residuals_after = [float(x.cpu()) for x in residuals_after]
      residuals1a[group][before] = (residuals_train, residuals_val, residuals_before, residuals_after)
    except:
      print(f"No residuals for {group} before {before}")
      residuals1a[group][before] = [[0],[0],[0],[0]]


In [ ]:
with open(f'{predir}/residuals/residuals1a.pkl', 'wb') as f:
  pickle.dump(residuals1a, f)

# split 2

In [ ]:
date_policy2 = pd.to_datetime('2025-01-07')
earliest_before = pd.to_datetime('2024-09-01')

before_dates = pd.date_range(start=earliest_before - pd.Timedelta(days=1), end=date_policy2 - pd.Timedelta(days=14), freq='1ME') + pd.Timedelta(days=1)
len(before_dates), before_dates

In [ ]:
models2 = dict()
residuals2 = dict()

data_after = data[data['date'] >= date_policy2]
# for group in tqdm(group_order):
for group in ['queer']:
  data_after_group = data_after[data_after['group'] == group]

  data_group = data[data['group'] == group]
  for before in before_dates:
    data_before_group = data_group[data_group['date'] > date_policy1]
    data_before_group = data_group[data_group['date'] < before]
    if len(data_before_group) < 100:
      print(f"Not enough data for {group} before {before}")
    else:
      # dataloaders
      train_loader, val_loader = df_to_traintest(data_before_group, TwoBranchPrepData)
      after_loader = data_loader(data_after_group, TwoBranchPrepData)

      # train
      model = TwoBranchLikesPredictor(
          sbert_dim = 384, reachback_length=REACHBACK
      ).to(device)

      model, train_loss_history, val_loss_history = train_model(model, train_loader, val_loader, lr=0.001, num_epochs= 20, verbose = False)

      torch.save(model.state_dict(), f'{predir}/models2/{group}_{before}.pt')

      # get residuals
      model.eval()
      residuals_train, residuals_val, residuals_before, residuals_after = [], [], [], []
      with torch.no_grad():
          for inputs, targets in train_loader:
              inputs, targets = inputs.to(device), targets.to(device)
              outputs = model(inputs)
              residuals_train.extend(targets - outputs.squeeze())
              residuals_before.extend(targets - outputs.squeeze())

      with torch.no_grad():
          for inputs, targets in val_loader:
              inputs, targets = inputs.to(device), targets.to(device)
              outputs = model(inputs)
              residuals_val.extend(targets - outputs.squeeze())
              residuals_before.extend(targets - outputs.squeeze())

      with torch.no_grad():
          for inputs, targets in after_loader:
              inputs, targets = inputs.to(device), targets.to(device)
              outputs = model(inputs)
              residuals_after.extend(targets - outputs.squeeze())

      group_residuals = (residuals_train, residuals_val, residuals_before, residuals_after)
      with open(f'{predir}/residuals2/{group}_{before}.pkl', 'wb') as f:
        pickle.dump(group_residuals, f)


In [ ]:
for group in tqdm(group_order):
  for before in before_dates:
    try:
      residuals_train, residuals_val, residuals_before, residuals_after = residuals2[group][before]
      # move to cpu, change to numpy array
      # residuals_train = [float(x.cpu()) for x in residuals_train]
      # residuals_val = [float(x.cpu()) for x in residuals_val]
      # residuals_before = [float(x.cpu()) for x in residuals_before]
      # residuals_after = [float(x.cpu()) for x in residuals_after]
      residuals_train = [float(x) for x in residuals_train]
      residuals_val = [float(x) for x in residuals_val]
      residuals_before = [float(x) for x in residuals_before]
      residuals_after = [float(x) for x in residuals_after]
      residuals2[group][before] = (residuals_train, residuals_val, residuals_before, residuals_after)
    except:
      print(f"No residuals for {group} before {before}")
      residuals2[group][before] = [[0],[0],[0],[0]]

In [ ]:
with open(f'{predir}/residuals/residuals2.pkl', 'wb') as f:
  pickle.dump(residuals2, f)